# Tate Project in Python
Tasks:
1. Which artist in this data set lived the longest?
2. Who are the top 10 artists by the number of artworks?
3. Which artist is created the most artwork by total surface area?
4. Did any artists have artwork acquired during their lifetime?
5. Please review the quality of the data, and present any issues


### Prepare data
Data is cleaned in preparation for answering above questions

##### Import libraries and read data from db

In [1]:
import numpy as np 
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
from skimpy import clean_columns

path = "C:/Users/katba/Data analysis learning/Portfolio/tate_project/data/"
database = path + 'artists.db'

In [2]:
conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,artists,artists,2,"CREATE TABLE ""artists"" (\n""Artist ID"" INTEGER,..."
1,table,artworks,artworks,159,"CREATE TABLE ""artworks"" (\n""Artwork ID"" INTEGE..."


In [3]:
artists = pd.read_sql("""SELECT *
                        FROM artists;""", conn)

artists

,Artist ID,Name,Nationality,Gender,Birth Year,Death Year
0,1,Robert Arneson,American,Male,1930.0,1992.0
1,2,Doroteo Arnaiz,Spanish,Male,1936.0,NaN
2,3,Bill Arnold,American,Male,1941.0,NaN
3,4,Charles Arnoldi,American,Male,1946.0,NaN
4,5,Per Arnoldi,Danish,Male,1941.0,NaN
...,...,...,...,...,...,...
15086,67452,Liu Jianhua,Chinese,Male,1962.0,NaN
15087,67453,Leng Lin,Chinese,Male,1965.0,NaN
15088,67652,Ellie Nagler,None,None,NaN,NaN
15089,67694,Glenn Williams,None,Male,NaN,NaN


In [4]:
artworks = pd.read_sql("""SELECT *
                        FROM artworks;""", conn)

artworks

,Artwork ID,Title,Artist ID,Name,Date,Medium,Dimensions,Acquisition Date,Credit,Catalogue,...,Classification,Object Number,Diameter (cm),Circumference (cm),Height (cm),Length (cm),Width (cm),Depth (cm),Weight (kg),Duration (s)
0,2,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",6210,Otto Wagner,1896,Ink and cut-and-pasted painted pages on paper,"19 1/8 x 66 1/2"" (48.6 x 168.9 cm)",1996-04-09,Fractional and promised gift of Jo Carole and ...,Y,...,Architecture,885.1996,NaN,NaN,48.6000,NaN,168.9000,NaN,NaN,NaN
1,3,"City of Music, National Superior Conservatory ...",7470,Christian de Portzamparc,1987,Paint and colored pencil on print,"16 x 11 3/4"" (40.6 x 29.8 cm)",1995-01-17,Gift of the architect in honor of Lily Auchinc...,Y,...,Architecture,1.1995,NaN,NaN,40.6401,NaN,29.8451,NaN,NaN,NaN
2,4,"Villa near Vienna Project, Outside Vienna, Aus...",7605,Emil Hoppe,1903,"Graphite, pen, color pencil, ink, and gouache ...","13 1/2 x 12 1/2"" (34.3 x 31.8 cm)",1997-01-15,Gift of Jo Carole and Ronald S. Lauder,Y,...,Architecture,1.1997,NaN,NaN,34.3000,NaN,31.8000,NaN,NaN,NaN
3,5,"The Manhattan Transcripts Project, New York, N...",7056,Bernard Tschumi,1980,Photographic reproduction with colored synthet...,"20 x 20"" (50.8 x 50.8 cm)",1995-01-17,Purchase and partial gift of the architect in ...,Y,...,Architecture,2.1995,NaN,NaN,50.8000,NaN,50.8000,NaN,NaN,NaN
4,6,"Villa, project, outside Vienna, Austria, Exter...",7605,Emil Hoppe,1903,"Graphite, color pencil, ink, and gouache on tr...","15 1/8 x 7 1/2"" (38.4 x 19.1 cm)",1997-01-15,Gift of Jo Carole and Ronald S. Lauder,Y,...,Architecture,2.1997,NaN,NaN,38.4000,NaN,19.1000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130257,217983,Seul/NY/MAX,4469,Nam June Paik,None,VHS,None,2008-10-08,The Gilbert and Lila Silverman Fluxus Collecti...,N,...,Media,FC5038,NaN,NaN,0.0000,NaN,0.0000,0.0,NaN,NaN
130258,217984,Fluxus-Manifestatie in en Rond Kunsthandel Monet,None,None,1962,VHS,None,2008-10-08,The Gilbert and Lila Silverman Fluxus Collecti...,N,...,Media,FC5039,NaN,NaN,0.0000,NaN,0.0000,0.0,NaN,NaN
130259,217985,Fluxphone Compositions,67695,Ely Ramen,1969,Cassette,None,2008-10-08,The Gilbert and Lila Silverman Fluxus Collecti...,N,...,Media,FC5040,NaN,NaN,0.0000,NaN,0.0000,0.0,NaN,NaN
130260,217986,Unidentified,None,None,None,Cassette,None,2008-10-08,The Gilbert and Lila Silverman Fluxus Collecti...,N,...,Media,FC5041,NaN,NaN,0.0000,NaN,0.0000,0.0,NaN,NaN


Use info to see overall information about artists dataframe\
Observations: 
* Null values for: Nationality, Gender, Birth Year, Death Year
* Column names do not follow conventions
* Birth Year and Death Year are float datatypes

In [5]:
artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15091 entries, 0 to 15090
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Artist ID    15091 non-null  int64  
 1   Name         15091 non-null  object 
 2   Nationality  12603 non-null  object 
 3   Gender       12019 non-null  object 
 4   Birth Year   11237 non-null  float64
 5   Death Year   4579 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 707.5+ KB


Change column names

In [6]:
artists = clean_columns(artists)

Change birth_year and death_year type to dates

In [7]:
artists.birth_year.unique()
artists.death_year.unique()

array([1992.,   nan, 1966., 1995., 1997., 1964., 2013., 1942., 1941.,
       1940., 2000., 1976., 1949., 2008., 1973., 1953., 1991., 1971.,
       2010., 2011., 2007., 1994., 1967., 1881., 1984., 2002., 1951.,
       1988., 1987., 1978., 1993., 1894., 1999., 1975., 1996., 1969.,
       2012., 1983., 1989., 1959., 2003., 1936., 2006., 1961., 1998.,
       1980., 1982., 1877., 1956., 1879., 1946., 1887., 1985., 2005.,
       1927., 1952., 1871., 1931., 1960., 1948., 2009., 2004., 1965.,
       1955., 2001., 1981., 1990., 1924., 1986., 1968., 1882., 1958.,
       2014., 2016., 1972., 1944., 1905., 1938., 1902., 1934., 1979.,
       1963., 2015., 1954., 1898., 1903., 1950., 1922., 1893., 1910.,
       1925., 1908., 1962., 1957., 1900., 1909., 1977., 1828., 1933.,
       1870., 1876., 1896., 1916., 1930., 1970., 1885., 1947., 1943.,
       1929., 1912., 1919., 1937., 1945., 1939., 1890., 1906., 1926.,
       1856., 1932., 1863., 1974., 1875., 1913., 1892., 1878., 1928.,
       1901., 1920.,

In [8]:
artists['birth_year'] = pd.to_datetime(artists['birth_year'], format='%Y')
artists['death_year'] = pd.to_datetime(artists['death_year'], format='%Y')
artists.head()

,artist_id,name,nationality,gender,birth_year,death_year
0,1,Robert Arneson,American,Male,1930-01-01,1992-01-01
1,2,Doroteo Arnaiz,Spanish,Male,1936-01-01,NaT
2,3,Bill Arnold,American,Male,1941-01-01,NaT
3,4,Charles Arnoldi,American,Male,1946-01-01,NaT
4,5,Per Arnoldi,Danish,Male,1941-01-01,NaT


Check for duplicates

In [9]:
artists.duplicated().sum()

0

Strip all leading or trailing whitespace

In [10]:
cols = artists.select_dtypes(object).columns
artists[cols] = artists[cols].apply(lambda x: x.str.strip())

Check unique values for nationality and gender

In [11]:
artists.gender.unique()

array(['Male', 'Female', None, 'male'], dtype=object)

In [12]:
artists['gender'].replace(['male'],['Male'], inplace=True)
artists.gender.unique()

array(['Male', 'Female', None], dtype=object)

In [13]:
artists.nationality.unique()

array(['American', 'Spanish', 'Danish', 'Italian', 'French', 'Estonian',
       'Mexican', 'Swedish', None, 'Israeli', 'British', 'Finnish',
       'Polish', 'Japanese', 'Guatemalan', 'Colombian', 'Romanian',
       'Russian', 'German', 'Argentine', 'Kuwaiti', 'Various', 'Belgian',
       'Dutch', 'Norwegian', 'Nationality unknown', 'Chilean', 'Swiss',
       'Costa Rican', 'Czech', 'Brazilian', 'Austrian', 'Canadian',
       'Australian', 'Ukrainian', 'Hungarian', 'Haitian', 'Congolese',
       'Bolivian', 'Cuban', 'Yugoslav', 'Portuguese', 'Indian',
       'Icelandic', 'Irish', 'Guyanese', 'Uruguayan', 'Slovak',
       'Croatian', 'Greek', 'Peruvian', 'Chinese', 'Venezuelan',
       'Turkish', 'Panamanian', 'Algerian', 'Ecuadorian', 'South African',
       'Iranian', 'Korean', 'Canadian Inuit', 'Paraguayan',
       'Luxembourgish', 'Nicaraguan', 'Zimbabwean', 'Moroccan',
       'Tanzanian', 'Bulgarian', 'Tunisian', 'Sudanese', 'Taiwanese',
       'Ethiopian', 'Slovenian', 'Scottish',

Reviewing this will be easier in alphabetical order. Sorted function will not work with nan value present, therefore nan values are replaced with 'Nationality Unknown'.

In [14]:
artists.nationality.fillna('Nationality Unknown', inplace=True)
nat = artists.nationality.unique()
sorted(nat)

['Afghan',
 'Albanian',
 'Algerian',
 'American',
 'Angolan',
 'Argentine',
 'Australian',
 'Austrian',
 'Azerbaijani',
 'Bahamian',
 'Belgian',
 'Bolivian',
 'Bosnian',
 'Brazilian',
 'British',
 'Bulgarian',
 'Burkinabe',
 'Cambodian',
 'Cameroonian',
 'Canadian',
 'Canadian Inuit',
 'Catalan',
 'Chilean',
 'Chinese',
 'Colombian',
 'Congolese',
 'Costa Rican',
 'Croatian',
 'Cuban',
 'Cypriot',
 'Czech',
 'Czechoslovakian',
 'Danish',
 'Dutch',
 'Ecuadorian',
 'Egyptian',
 'Emirati',
 'Estonian',
 'Ethiopian',
 'Filipino',
 'Finnish',
 'French',
 'Georgian',
 'German',
 'Ghanaian',
 'Greek',
 'Guatemalan',
 'Guyanese',
 'Haitian',
 'Hungarian',
 'Icelandic',
 'Indian',
 'Indonesian',
 'Iranian',
 'Iraqi',
 'Irish',
 'Israeli',
 'Italian',
 'Ivorian',
 'Japanese',
 'Kazakhstani',
 'Kenyan',
 'Korean',
 'Kuwaiti',
 'Kyrgyzstani',
 'Latvian',
 'Lebanese',
 'Lithuanian',
 'Luxembourgish',
 'Macedonian',
 'Malaysian',
 'Malian',
 'Mauritanian',
 'Mexican',
 'Moroccan',
 'Mozambican',
 'N

No duplicate countries with different spellings. Three different spellings of 'Nationality Unknown' which are now replaced. 

In [15]:
artists['nationality'].replace({'nationality unknown':'Nationality Unknown', 'Nationality unknown':'Nationality Unknown'}, inplace=True)

Check max and min dates for sensible values.

In [16]:
artists.describe()

,artist_id,birth_year,death_year
count,15091.000000,11237,4579
mean,18297.556027,1930-11-08 12:23:46.288155136,1974-04-15 21:00:07.075780736
min,1.000000,1730-01-01 00:00:00,1795-01-01 00:00:00
25%,4195.500000,1910-01-01 00:00:00,1958-01-01 00:00:00
50%,8593.000000,1936-01-01 00:00:00,1980-01-01 00:00:00
75%,33088.500000,1956-01-01 00:00:00,1998-01-01 00:00:00
max,67695.000000,2012-01-01 00:00:00,2017-01-01 00:00:00
std,16632.963898,NaN,NaN


Create calculated column for age to check difference between birth and death is sensible.   


In [17]:
artists['age'] = (artists['death_year'] - artists['birth_year'])

In [18]:
artists.describe()

,artist_id,birth_year,death_year,age
count,15091.000000,11237,4579,4566
mean,18297.556027,1930-11-08 12:23:46.288155136,1974-04-15 21:00:07.075780736,26416 days 07:14:54.086727680
min,1.000000,1730-01-01 00:00:00,1795-01-01 00:00:00,365 days 00:00:00
25%,4195.500000,1910-01-01 00:00:00,1958-01-01 00:00:00,23376 days 00:00:00
50%,8593.000000,1936-01-01 00:00:00,1980-01-01 00:00:00,27393 days 00:00:00
75%,33088.500000,1956-01-01 00:00:00,1998-01-01 00:00:00,30316 days 00:00:00
max,67695.000000,2012-01-01 00:00:00,2017-01-01 00:00:00,47481 days 00:00:00
std,16632.963898,NaN,NaN,5825 days 05:16:02.512164864


Max value for age is 130 years. Oldest person ever alive was 122 so this is not possible. View all rows where age is larger than possible. 

In [19]:
artists[artists['age']>'44530 days']

,artist_id,name,nationality,gender,birth_year,death_year,age
12486,38294,"Union Paper Bag Machine Company, Philadelphia, PA",Nationality Unknown,None,1869-01-01,1999-01-01,47481 days


'Union Paper Bag Machine Company' is not an artist, this row can be dropped.

In [20]:
artists.drop(12486, inplace=True)

Use info to see overall information about artists dataframe\
Observations: 
* Null values for: Artist ID, Name, Date, Medium, Dimensions, Acquisition date, credit, all dimensions
* Column names not using convention
* Artist Id is object - should be int64
* Date and aquisition date are object - should be datetime

In [21]:
artworks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130262 entries, 0 to 130261
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Artwork ID          130262 non-null  int64  
 1   Title               130210 non-null  object 
 2   Artist ID           128802 non-null  object 
 3   Name                128802 non-null  object 
 4   Date                127954 non-null  object 
 5   Medium              118343 non-null  object 
 6   Dimensions          118799 non-null  object 
 7   Acquisition Date    124799 non-null  object 
 8   Credit              127192 non-null  object 
 9   Catalogue           130262 non-null  object 
 10  Department          130262 non-null  object 
 11  Classification      130262 non-null  object 
 12  Object Number       130262 non-null  object 
 13  Diameter (cm)       1399 non-null    float64
 14  Circumference (cm)  10 non-null      float64
 15  Height (cm)         111893 non-nul

Change column names

In [22]:
artworks = clean_columns(artworks)

Change data types for aquisition_date

In [23]:
artworks['acquisition_date'].unique()

array(['1996-04-09', '1995-01-17', '1997-01-15', ..., '1216-10-18',
       '2016-10', '2016-05'], dtype=object)

In [24]:
artworks['acquisition_date'] = pd.to_datetime(artworks['acquisition_date'], format='mixed', errors='coerce')


Check for duplicates

In [25]:
artworks.duplicated().sum()

0

Strip leading/trailing whitespace

In [26]:
cols2 = artists.select_dtypes(object).columns
artists[cols2] = artists[cols2].apply(lambda x: x.str.strip())

Check unique values for classification and catalogue

In [27]:
artworks['classification'].unique()

array(['Architecture', 'Mies van der Rohe Archive', 'Design',
       'Illustrated Book', 'Print', 'Drawing', 'Film', 'Multiple',
       'Periodical', 'Photograph', 'Painting', 'Product Design',
       'Photography Research/Reference', 'Media', 'Sculpture', 'Textile',
       'Installation', 'Video', 'Work on Paper', 'Audio', 'Performance',
       '(not assigned)', 'Film (object)', 'Ephemera', 'Collage',
       'Frank Lloyd Wright Archive', 'Furniture and Interiors',
       'Software'], dtype=object)

In [28]:
artworks['catalogue'].unique()

array(['Y', 'N'], dtype=object)

Change type for artist_id to int. 
This is not straightforward due to the format of artist_id being a string of ids separated by a comma, as below.

In [30]:
artist_id_comma_only = artworks[artworks['artist_id'].str.contains(',', na=False)]
artist_id_comma_only

,artwork_id,title,artist_id,name,date,medium,dimensions,acquisition_date,credit,catalogue,...,classification,object_number,diameter_cm,circumference_cm,height_cm,length_cm,width_cm,depth_cm,weight_kg,duration_s
65,81,"House IV Transformation Study, project, Falls ...","6969, 8134","Peter Eisenman, Robert Cole",1975,Ink and color ink on frosted polymer sheet,"13 3/4 x 44 5/8"" (34.9 x 113.3 cm)",1980-01-08,Gift of Philip Johnson,Y,...,Architecture,20.198,NaN,NaN,34.9251,NaN,113.3477,NaN,NaN,NaN
66,82,"Villa dall'Ava, Paris (Saint-Cloud), France, E...","6956, 6957","Rem Koolhaas, Madelon Vriesendorp",1987,Synthetic polymer paint and ink on paper,"25 x 39"" (63.5 x 99.1 cm)",2000-01-19,Gift of Max Underwood,Y,...,Architecture,21.2,NaN,NaN,63.5001,NaN,99.0602,NaN,NaN,NaN
76,100,"Regional Administrative Center, project ""Tries...","7661, 8131, 8180","Aldo Rossi, Gianni Braghieri, M. Bosshard",1974,Rubbed ink and pastel on whiteprint,"28 1/2 x 36"" (72.4 x 91.4 cm)",1980-01-08,Philip Johnson Fund,Y,...,Architecture,28.198,NaN,NaN,72.4000,NaN,91.4000,NaN,NaN,NaN
107,148,"Woodland Crematorium, Woodland Cemetery, Stock...","27, 24452","Erik Gunnar Asplund, Sigurd Lewerentz",1937,Graphite on tracing paper,"16 1/4 x 37 7/8"" (41.3 x 96.2 cm)",1990-01-17,"Gift of Blanchette Hooker Rockefeller, Mrs. Gi...",Y,...,Architecture,62.199,NaN,NaN,41.3000,NaN,96.2000,NaN,NaN,NaN
110,151,"Palais de la Découverte Project, Paris, France...","8102, 6703, 4312","Paul Nelson, Frantz Jourdain, Oscar Nitzchke",1938,Ink and color pencil on paper mounted on board,"14 3/4 x 37 1/2 x 1/2"" (37.5 x 95.3 x 1.3 cm)",1966-01-01,Gift of the architects,Y,...,Architecture,98.2013,NaN,NaN,37.5000,NaN,95.3000,1.3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130133,216775,Dear Data,"67122, 39150","Giorgia Lupi, Stefanie Posavec",2015,Hand-illustrated postcards,None,2016-10-17,Committee on Architecture and Design Funds,N,...,Design,962.2016.50a-b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130134,216776,Dear Data,"67122, 39150","Giorgia Lupi, Stefanie Posavec",2015,Hand-illustrated postcards,None,2016-10-17,Committee on Architecture and Design Funds,N,...,Design,962.2016.51a-b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130135,216777,Dear Data,"67122, 39150","Giorgia Lupi, Stefanie Posavec",2015,Hand-illustrated postcards,None,2016-10-17,Committee on Architecture and Design Funds,N,...,Design,962.2016.52a-b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130239,217965,Misc performances 1959-1960,"2637, 30644","Dick Higgins, Al Hansen",None,Cassette,None,2008-10-08,The Gilbert and Lila Silverman Fluxus Collecti...,N,...,Media,FC5020,NaN,NaN,0.0000,NaN,0.0000,0.0,NaN,NaN


Create new df to work with manipulating artist_id type

In [177]:
artworks_artist_id = artworks

Drop rows where artist_id is null to allow split function to work.
Split on commas to create list, then convert values within list to int type.

In [176]:
artworks_artist_id.dropna(axis=0, subset='artist_id', inplace=True)

In [33]:
# str.split turns each value into a list, split on commas. apply calls the result of split into the lambda function. map turns each individual string within list into an int. list returns a list of these values.
artworks_artist_id['artist_id'] = artworks_artist_id['artist_id'].str.split(',').apply(lambda ids: list(map(int, ids)))

Check this has been successful

In [34]:
artworks_artist_id['artist_id']

0          [6210]
1          [7470]
2          [7605]
3          [7056]
4          [7605]
           ...   
130254     [2637]
130256     [4469]
130257     [4469]
130259    [67695]
130261    [21398]
Name: artist_id, Length: 128802, dtype: object

In [36]:
def checkType(a_list):
    for element in a_list:
        if isinstance(element, int):
            print("Int")
        else:
            print("Not int")

checkType(artworks_artist_id['artist_id'][0])

Int


Use explode function to replicate rows with more than one artist_id, to ensure there is a single artist_id per row.

In [37]:
artworks_artist_id_exploded = artworks_artist_id.explode('artist_id', ignore_index=True)
artworks_artist_id_exploded

,artwork_id,title,artist_id,name,date,medium,dimensions,acquisition_date,credit,catalogue,...,classification,object_number,diameter_cm,circumference_cm,height_cm,length_cm,width_cm,depth_cm,weight_kg,duration_s
0,2,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",6210,Otto Wagner,1896,Ink and cut-and-pasted painted pages on paper,"19 1/8 x 66 1/2"" (48.6 x 168.9 cm)",1996-04-09,Fractional and promised gift of Jo Carole and ...,Y,...,Architecture,885.1996,NaN,NaN,48.6000,NaN,168.9000,NaN,NaN,NaN
1,3,"City of Music, National Superior Conservatory ...",7470,Christian de Portzamparc,1987,Paint and colored pencil on print,"16 x 11 3/4"" (40.6 x 29.8 cm)",1995-01-17,Gift of the architect in honor of Lily Auchinc...,Y,...,Architecture,1.1995,NaN,NaN,40.6401,NaN,29.8451,NaN,NaN,NaN
2,4,"Villa near Vienna Project, Outside Vienna, Aus...",7605,Emil Hoppe,1903,"Graphite, pen, color pencil, ink, and gouache ...","13 1/2 x 12 1/2"" (34.3 x 31.8 cm)",1997-01-15,Gift of Jo Carole and Ronald S. Lauder,Y,...,Architecture,1.1997,NaN,NaN,34.3000,NaN,31.8000,NaN,NaN,NaN
3,5,"The Manhattan Transcripts Project, New York, N...",7056,Bernard Tschumi,1980,Photographic reproduction with colored synthet...,"20 x 20"" (50.8 x 50.8 cm)",1995-01-17,Purchase and partial gift of the architect in ...,Y,...,Architecture,2.1995,NaN,NaN,50.8000,NaN,50.8000,NaN,NaN,NaN
4,6,"Villa, project, outside Vienna, Austria, Exter...",7605,Emil Hoppe,1903,"Graphite, color pencil, ink, and gouache on tr...","15 1/8 x 7 1/2"" (38.4 x 19.1 cm)",1997-01-15,Gift of Jo Carole and Ronald S. Lauder,Y,...,Architecture,2.1997,NaN,NaN,38.4000,NaN,19.1000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142276,217980,"Requiem for Wagner The Criminal Mayor - ""Eggs""...",2637,Dick Higgins,None,None,None,2008-10-08,The Gilbert and Lila Silverman Fluxus Collecti...,N,...,Media,FC5035,NaN,NaN,0.0000,NaN,0.0000,0.0,NaN,NaN
142277,217982,Study No. 3,4469,Nam June Paik,None,VHS,None,2008-10-08,The Gilbert and Lila Silverman Fluxus Collecti...,N,...,Media,FC5037,NaN,NaN,0.0000,NaN,0.0000,0.0,NaN,NaN
142278,217983,Seul/NY/MAX,4469,Nam June Paik,None,VHS,None,2008-10-08,The Gilbert and Lila Silverman Fluxus Collecti...,N,...,Media,FC5038,NaN,NaN,0.0000,NaN,0.0000,0.0,NaN,NaN
142279,217985,Fluxphone Compositions,67695,Ely Ramen,1969,Cassette,None,2008-10-08,The Gilbert and Lila Silverman Fluxus Collecti...,N,...,Media,FC5040,NaN,NaN,0.0000,NaN,0.0000,0.0,NaN,NaN


In [38]:
artworks_artist_id_exploded['artist_id'] = artworks_artist_id_exploded['artist_id'].astype(str).astype(int)

In [39]:
artworks_artist_id_exploded['artist_id']

0          6210
1          7470
2          7605
3          7056
4          7605
          ...  
142276     2637
142277     4469
142278     4469
142279    67695
142280    21398
Name: artist_id, Length: 142281, dtype: int32

## Task 1: Which artist in this data set lived the longest?
#### Answer:
* Leni Matthaei and B. Efimov both lived for 108 years. 
* One value which 'lived' for longer (130 years) has been discounted due to it being a company rather than artist. 

Use max function to find largest age, then df filter by this value.

In [40]:
artists['age'].max()

Timedelta('39446 days 00:00:00')

In [41]:
artists[artists['age']=='39446 days']

,artist_id,name,nationality,gender,birth_year,death_year,age
3449,3844,Leni Matthaei,German,Female,1873-01-01,1981-01-01,39446 days
9164,23931,B. Efimov,Russian,Male,1900-01-01,2008-01-01,39446 days


## Task 2: Who are the top 10 artists by the number of artworks?

#### Answer:
* Top 10 included 'Unknown photographer' which should be ignored
* See dataframe table below

Use value_counts to find the artist_ids which occur most frequently (have the most pieces of art).\
head(11) rather than head(10) ammended as one row returns 'unknown', this is later removed.

In [67]:
top_10 = artworks_artist_id_exploded['artist_id'].value_counts().head(11)

artist_id
229      5050
710      3363
7166     2595
8595     1656
1633     1436
2002     1335
4609     1323
1055     1174
3832     1069
665       909
21398     903
Name: count, dtype: int64

In [92]:
top_10.to_frame()

,count
artist_id,
229,5050
710,3363
7166,2595
8595,1656
1633,1436
2002,1335
4609,1323
1055,1174
3832,1069


Merge top_10 with artists df to view name (and other data) for the top ten artists./
'Unknown' dropped at this point.

In [109]:
top_10_artists = pd.merge(top_10, artists, how='left', on='artist_id', sort=False)

In [108]:
top_10_artists = top_10_artists.rename(columns=({'count':'number_artworks'}))
top_10_artists = top_10_artists.drop(labels=3)
top_10_artists

,artist_id,number_artworks,name,nationality,gender,birth_year,death_year,age
0,229,5050,Eugène Atget,French,Male,1857-01-01,1927-01-01,25566 days
1,710,3363,Louise Bourgeois,American,Female,1911-01-01,2010-01-01,36160 days
2,7166,2595,Ludwig Mies van der Rohe,American,Male,1886-01-01,1969-01-01,30315 days
4,1633,1436,Jean Dubuffet,French,Male,1901-01-01,1985-01-01,30681 days
5,2002,1335,Lee Friedlander,American,Male,1934-01-01,NaT,NaT
6,4609,1323,Pablo Picasso,Spanish,Male,1881-01-01,1973-01-01,33602 days
7,1055,1174,Marc Chagall,French,Male,1887-01-01,1985-01-01,35794 days
8,3832,1069,Henri Matisse,French,Male,1869-01-01,1954-01-01,31045 days
9,665,909,Pierre Bonnard,French,Male,1867-01-01,1947-01-01,29219 days
10,21398,903,George Maciunas,American,Male,1931-01-01,1978-01-01,17167 days


## Task 3: Which artist is created the most artwork by total area?

#### Answer:
* James Lee Byars

Create calculated column for area, using height and width columns

In [116]:
artworks_artist_id_exploded['area_cm2'] = artworks_artist_id_exploded['height_cm'] * artworks_artist_id_exploded['width_cm']

Calculate total area for each artist, sort from largest to smallest, select only the top value.

In [131]:
total_areas = artworks_artist_id_exploded.groupby('artist_id')['area_cm2'].sum()
total_areas = total_areas.sort_values(ascending=False)
top_total_area = total_areas.head(1)

Merge with artists df to display name associated with the artist_id

In [133]:
top_total_area.to_frame()

,area_cm2
artist_id,
902,1.035262e+07


In [136]:
top_area_artist = pd.merge(top_total_area, artists, how='left', on='artist_id')
pd.options.display.float_format = '{:.0f}'.format
top_area_artist

,artist_id,area_cm2,name,nationality,gender,birth_year,death_year,age
0,902,10352623,James Lee Byars,American,Male,1932-01-01,1997-01-01,23742 days


## Task 4: Did any artists have artwork acquired during their lifetime?

#### Answer:
* Yes - 9143 artists had artwork acquired before their death
* Caveat: assumption has been made that presence of birth date with no death date means the artist is still alive

Given the question requires knowledge of lifetime, drop columns where both birth and death year are NA. 

In [143]:
merged_df.dropna(axis=0, how='all', subset=['birth_year', 'death_year'], inplace=True)

Create new columns using conditional logic to determine whether work was acquired during lifetime. Two categories for artists which are dead and assumed alive.

In [149]:
merged_df['acquired_before_died'] = np.where(merged_df['acquisition_date'] < merged_df['death_year'], True, False)

In [163]:
merged_df['acquired_still_alive'] = np.where((merged_df['acquisition_date'] > merged_df['birth_year']) & (merged_df['death_year'].isna()), True, False)

Count how many artworks were acquired during lifetime using the two conditions

In [154]:
merged_df['acquired_before_died'].value_counts()

acquired_before_died
False    95430
True     35779
Name: count, dtype: int64

In [171]:
merged_df['acquired_still_alive'].value_counts()

acquired_still_alive
False    85203
True     46006
Name: count, dtype: int64

Count how many individual artists this corresponds to

In [175]:
acquired_while_alive = merged_df[(merged_df['acquired_before_died']== True) | (merged_df['acquired_still_alive']== True)]
acquired_while_alive['artist_id'].nunique()


9143

## Task 5: Please review the quality of the data, and present any issues

#### Answer:
* The data has a high number of null values. Care was taken to minimise dropping null values, but on occasion this was required. This means not all pieces of artwork were included in analysis.
* The database design was flawed in terms of its representation of the many to many relationships between artists and artworks. The database included multiple artist IDs for some pieces of art. Ideally the database would account for this relationship using a joining table of artwork and artist Ids only. A workaround was used for this analysis where 'explode' was used to create new artwork rows for each artist id in the artwork table. This is not an ideal solution as it means the artwork id was no longer unique. 
* In the artwork table the 'date' column contained a number of different date formats, including date ranges, this meant this data could not be used for date analysis without a significant amount of cleaning (not completed in this analysis.)